# Decorators
https://realpython.com/primer-on-python-decorators/

A syntax for calling **higher-order functions** (a function which takes another function as argument and/or returns a function)

A decorators is a function which takes a function and extends its behaviour without explicitly modifying the function itself.

Think of functions simplistically as machines which return a value based on given argument.

There are 3 properties of functions which allow us to use decorators (then a bit of Syntax)

## Function are objects: the consequences
### Functions are objects, and can be passed as arguments 
In Python, functions are **first class objects**. That means they act like more traditional objects - importantly for this tutorial they can be passed around as arguments

In [1]:
# passing around functions as arguments to other functions
def say_hello(name):
    return f"Hello {name}"

def say_ahoy(name):
    return f"Ahoyhoy {name}"

def greet_bob(greeter_func): # note func passed without (), referring to the func as an object
    return greeter_func("Bob") # func passed with (), calling the function

In [2]:
greet_bob(say_hello)

'Hello Bob'

In [3]:
greet_bob(say_ahoy)

'Ahoyhoy Bob'

### Functions can be defined inside other functions ('Inner Functions')

In [4]:
# example of an inner function
def parent():
    print("printing froom parent")
    def child1():
        print("printing from child1")
    def child2():
        print("printing from child2")
    
    child2()
    child1()
    
parent()

printing froom parent
printing from child2
printing from child1


Note you can't actually run `child1` on its own, it is *locally scoped* to `parent`.  

The `childx` functions are not defined until `parent()` is called.

### Functions returning functions

In [5]:
# function returning function
def parent(num):
    def child1():
        return "printing from child1"
    def child2():
        return "printing from child2"
    
    if num == 1:
        return child1
    else:
        return child2

In [6]:
parent(1) # returns a FUNCTION

<function __main__.parent.<locals>.child1()>

In [7]:
parent(1)() # parent returns child1 and then child1 is called

'printing from child1'

In [8]:
# prettier way to do it.
first = parent(1)
second = parent(0)

first()

'printing from child1'

## Simple decorators

An example

In [9]:
# simple decorator
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_hello():
    print("Hello!")
    
print('say_hello name:',say_hello.__name__,'\n')

say_hello = my_decorator(say_hello)
say_hello()

print('\nsay_hello name:',say_hello.__name__)

say_hello name: say_hello 

Something is happening before the function is called.
Hello!
Something is happening after the function is called.

say_hello name: wrapper


We can see this is an application of the 3 properties of functions discussed above:
1. `my_decorator` accepts a function as an argument, and in line 14 we pass the `say_hello` function to it.
2. `wrapper` is an inner function, which does something, runs the passed function, and then does something else
3. `my_decorator` returns the `wrapper` function.

So in line 14, `say_hello` is 'decorated', i.e. set to the wrapper, which does something, the runs `say_hello`, then does something else.

So decoraters wrap a function, modifying its behaviour without modifying the function. 

`say_hello = my_decorator(say_hello)` is replaced with 'Pie' Syntax

In [10]:
@my_decorator
def say_hello():
    print("Hello!")
    
say_hello()

Something is happening before the function is called.
Hello!
Something is happening after the function is called.


## Decorators with arguments
The above example is simple because the function we used it on had no arguments - rare in real life. The below shows you how to implement argments in a generalised way

In [11]:
# The decorator
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

In [12]:
# the function to wrap
@do_twice
def greet(name):
    print(f"Hello {name}")

In [13]:
greet('bob')

Hello bob
Hello bob


## Decorating functions with return values
Most functions will have a return, but the above pattern runs into issues with that - it 'eats' the return value

In [14]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [15]:
x = return_greeting('bob')

Creating greeting
Creating greeting


In [16]:
print(x)

None


To fix this we need to make sure the wrapper function returns the value from the original function

In [17]:
# The decorator with a return for the wrapper
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [18]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [19]:
x = return_greeting('bob')

Creating greeting
Creating greeting


In [20]:
print(x)

Hi bob


We saw before that a decorated function gets 'confused' about what it is

In [21]:
return_greeting.__name__ # it thinks its the wrapper!

'wrapper_do_twice'

It is technically true, but it is unhelpful. The way to fix it is with `@functools.wraps`.

In [22]:
# decorator with preserved attributes
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [23]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [24]:
return_greeting.__name__

'return_greeting'

## In the real world

Boilerplate for a decorator function:

In [25]:
# Boilerplate Decorator
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

### Timer decorator

In [26]:
# timer decorator
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

In [27]:
@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [28]:
waste_some_time(1000)

Finished 'waste_some_time' in 3.6440 secs


### Debugger Decorator



In [29]:
# debugger decorator
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

In [30]:
# Apply a decorator to a standard library function
import math
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

In [31]:
approximate_e(terms=10)

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24
Calling factorial(5)
'factorial' returned 120
Calling factorial(6)
'factorial' returned 720
Calling factorial(7)
'factorial' returned 5040
Calling factorial(8)
'factorial' returned 40320
Calling factorial(9)
'factorial' returned 362880


2.7182815255731922

### Non-wrapping decorator

In [32]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

In [33]:
PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

In [34]:
randomly_greet('Kim')

Using 'be_awesome'


'Yo Kim, together we are the awesomest!'

In [35]:
globals()['say_hello']

<function __main__.say_hello(name)>

## Advanced Decorators
* class decorators
* multiple decorators
* decorators with arguments
* decorators with optional arguments
* stateful decorators
* classes AS decorators

### Class decorators
2 ways: 1st you can decorate the functions within a class. Some common builtin ones:

* `@classmethod` functions which return instances of the class
* `@staticmethod` not dependent on an actual instance of the class
* `@property` - a smarter way to do getting and setting

In [36]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        print('radius obtained using getter method')
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535

In [37]:
c = Circle(5)

The radius attribute and 2x radius methods create a getter and setter without actually having getter and setter syntax (e.g `c.set_radius(5)`.) So we get the benefits of getting and setting (like checking input) without actually changing how the code is written. To do this we have:

1. the actual attribute `_radius`, with the underscore preceding it
2. the `@property` decorator around `radius(self)`, the getter
3. the `@radius. setter` around `radius(self, value)`, the setter

In [38]:
c.radius

radius obtained using getter method


5

In [39]:
try:
    c.radius = -1
except:
    print("didn't work!")

didn't work!


We use `@property` to create a function that is called like an attribute. This effectively makes it an immutable property

In [40]:
c.area

radius obtained using getter method


78.5398163375

In [41]:
try:
    c.area = 100
except:
    print("didn't work!")

didn't work!


`@classmethod` `unit_circle` is passed `cls`, and returns an instance of the Circle class

In [42]:
c = Circle.unit_circle()
c.radius

radius obtained using getter method


1

`@staticmethod` doesn't need an instance of the class

In [43]:
c.pi()

3.1415926535

In [44]:
Circle.pi()

3.1415926535

The second way is to add the decorator to the class itself

In [45]:
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str